![Logo TUBAF](https://tu-freiberg.de/sites/default/files/media/freiberger-alumni-netzwerk-6127/wbm_orig_rgb_0.jpg)

Exercise material of the MSc-level course **Advanced Soil Mechanics**.
Held at Technische Universität Bergakademie Freiberg.

Comments to:

*Prof. Dr. Thomas Nagel  
Chair of Soil Mechanics and Foundation Engineering  
Geotechnical Institute  
Technische Universität Bergakademie Freiberg.*

https://tu-freiberg.de/en/soilmechanics

In [58]:
#HIDDEN
import numpy as np #numerical methods
#import sympy as sp #symbolic operations
import matplotlib.pyplot as plt #plotting
import sympy as sp
sp.init_printing(use_latex='mathjax') #makes sympy output look nice

#Some plot settings
import plot_functions.plot_settings

## Slope failure from a material point stability perspective

*Based on: Buscarnera and Di Prisco (2011)*

**Disclaimer: the following is not finished or checked, yet. Handle with care.**

We depart from the assumption of an infinite slope and are interested in the stress and strain evolution in a plane parallel to the slope surface (simple shear kinematics).

Furthermore, we assume the following set of equations to hold:

\begin{align}
    \dot{\sigma}_\text{n}' &= E_\text{s} \dot{\epsilon}_\text{el}
    \\
    \dot{\tau} &= G \dot{\gamma}_\text{el}
    \\
    f &= \tau - \sigma_\text{n}' \tan \varphi - c
    \\
    g &= \tau - \sigma_\text{n}' \tan \psi
\end{align}

The flow rule yields

\begin{align}
    \dot{\epsilon}_\text{pl} &= - \lambda \tan \psi
    \\
    \dot{\gamma}_\text{pl} &= \lambda
\end{align}

Consistency in the plastic state requires

\begin{align}
    \dot{f} = 0 = \dot{\tau} - \dot{\sigma}_\text{n}' \tan \varphi
\end{align}

Inserting the stress evolution

\begin{align}
    \dot{\sigma}_\text{n}' &= E_\text{s} \left( \dot{\epsilon} + \lambda \tan \psi \right)
    \\
    \dot{\tau} &= G \left( \dot{\gamma} - \lambda \right)
\end{align}

into the consistency condition yields

\begin{align}
    0 &= G \left( \dot{\gamma} - \lambda \right) - E_\text{s} \left( \dot{\epsilon} + \lambda \tan \psi \right) \tan \varphi
\end{align}

$$
    \lambda = \frac{G \dot{\gamma} - E_\text{s} \tan\varphi \dot{\epsilon}}{G + E_\text{s} \tan\varphi \tan\psi}
$$

In [59]:
lam, sign, tau, gamma, eps, E, G, phi, psi = sp.symbols('lambda \dot{\sigma}_\mathrm{n} \dot{\\tau} \dot{\gamma} \dot{\epsilon} E_\mathrm{s} G varphi psi')

In [60]:
Fs = sp.Eq(E * (eps + lam * sp.tan(psi)), sign)
Ft = sp.Eq(G*(gamma - lam),tau)
Fl = sp.Eq(G*(gamma - lam) - E*(eps+lam*sp.tan(psi))*sp.tan(phi),0)
display([Fs,Ft,Fl])

[E_\mathrm{s}⋅(\dot{\epsilon} + λ⋅tan(ψ)) = \dot{\sigma}_\mathrm{n}, G⋅(\dot{\
gamma} - λ) = \dot{\tau}, -E_\mathrm{s}⋅(\dot{\epsilon} + λ⋅tan(ψ))⋅tan(varphi
) + G⋅(\dot{\gamma} - λ) = 0]

In [61]:
result = sp.solve([Fs,Ft,Fl],(sign,tau,lam))
sp.simplify(result)

⎧                         E_\mathrm{s}⋅G⋅\dot{\epsilon} + E_\mathrm{s}⋅G⋅\dot{
⎨\dot{\sigma}_\mathrm{n}: ────────────────────────────────────────────────────
⎩                                        E_\mathrm{s}⋅tan(ψ)⋅tan(varphi) + G  

\gamma}⋅tan(ψ)              E_\mathrm{s}⋅G⋅\dot{\epsilon}⋅tan(varphi) + E_\mat
──────────────, \dot{\tau}: ──────────────────────────────────────────────────
                                                       E_\mathrm{s}⋅tan(ψ)⋅tan

hrm{s}⋅G⋅\dot{\gamma}⋅tan(ψ)⋅tan(varphi)     -E_\mathrm{s}⋅\dot{\epsilon}⋅tan(
────────────────────────────────────────, λ: ─────────────────────────────────
(varphi) + G                                            E_\mathrm{s}⋅tan(ψ)⋅ta

varphi) + G⋅\dot{\gamma}⎫
────────────────────────⎬
n(varphi) + G           ⎭

With this, the stress-strain relations in the elasto-plastic state reads

$$
    \begin{pmatrix}
    \dot{\sigma}_\text{n}' \\ \dot{\tau}
    \end{pmatrix}
    =
    \frac{E_\text{s} G}{E_\text{s}\tan\psi\tan\varphi + G} \begin{pmatrix}
    1 & \tan\psi
    \\
    \tan\varphi & \tan\varphi\tan\psi
    \end{pmatrix}
    \begin{pmatrix}
    \dot{\epsilon} \\ \dot{\gamma}
    \end{pmatrix}
$$

The system matrix is the control matrix for a drained slope.

In [62]:
D_drained = sp.Matrix([[1,sp.tan(psi)],[sp.tan(phi),sp.tan(phi)*sp.tan(psi)]]) * E*G/(E*sp.tan(psi)*sp.tan(phi)+G)
D_drained

⎡           E_\mathrm{s}⋅G                   E_\mathrm{s}⋅G⋅tan(ψ)       ⎤
⎢───────────────────────────────────  ───────────────────────────────────⎥
⎢E_\mathrm{s}⋅tan(ψ)⋅tan(varphi) + G  E_\mathrm{s}⋅tan(ψ)⋅tan(varphi) + G⎥
⎢                                                                        ⎥
⎢     E_\mathrm{s}⋅G⋅tan(varphi)       E_\mathrm{s}⋅G⋅tan(ψ)⋅tan(varphi) ⎥
⎢───────────────────────────────────  ───────────────────────────────────⎥
⎣E_\mathrm{s}⋅tan(ψ)⋅tan(varphi) + G  E_\mathrm{s}⋅tan(ψ)⋅tan(varphi) + G⎦

In [63]:
sp.det(D_drained)

0

The vanishing determinant indicates a loss of stability independent of the chosen material parameters and the tendency for the MC model to localize (elastic ideal plastic).

For the undrained case, we assume isochoric deformation and thus $\dot{\epsilon} = 0$ becomes a controlling variable in the present case. The system matrix is found by

In [64]:
result = sp.solve([Fs,Ft,Fl],(eps,tau,lam))
sp.simplify(result)

⎧                -E_\mathrm{s}⋅G⋅\dot{\gamma}⋅tan(ψ) + E_\mathrm{s}⋅\dot{\sigm
⎨\dot{\epsilon}: ─────────────────────────────────────────────────────────────
⎩                                                                      E_\math

a}_\mathrm{n}⋅tan(ψ)⋅tan(varphi) + G⋅\dot{\sigma}_\mathrm{n}                  
────────────────────────────────────────────────────────────, \dot{\tau}: \dot
rm{s}⋅G                                                                       

                                    G⋅\dot{\gamma} - \dot{\sigma}_\mathrm{n}⋅t
{\sigma}_\mathrm{n}⋅tan(varphi), λ: ──────────────────────────────────────────
                                                             G                

an(varphi)⎫
──────────⎬
          ⎭

$$
    \begin{pmatrix}
    \dot{\epsilon} \\ \dot{\tau}
    \end{pmatrix}
    =
    \begin{pmatrix}
    \frac{G + E_\text{s}\tan\psi\tan{\varphi}}{E_\text{s}G} & -\tan\psi
    \\
    \tan\varphi & 0
    \end{pmatrix}
    \begin{pmatrix}
    \dot{\sigma}_\text{n}' \\ \dot{\gamma}
    \end{pmatrix}
$$

In [65]:
D_undrained = sp.Matrix([[(G+E*sp.tan(phi)*sp.tan(psi))/(E*G),-sp.tan(psi)],[sp.tan(phi),0]])
D_undrained

⎡E_\mathrm{s}⋅tan(ψ)⋅tan(varphi) + G         ⎤
⎢───────────────────────────────────  -tan(ψ)⎥
⎢           E_\mathrm{s}⋅G                   ⎥
⎢                                            ⎥
⎣            tan(varphi)                 0   ⎦

In [66]:
sp.det(D_undrained)

tan(ψ)⋅tan(varphi)

This determinant vanishes for vanishing friction or dilatancy angles. The plausible solution in our case for a vanishing (or negative) second-order work is thus the transition to negative dilatancy angles, i.e. contractile material behaviour. For positive dilatancy the condition indicates stability.

Note in passing: this determinant corresponds to $D_{22} = 0$.